### CWT 변환 후 LSTM - AutoEncoder 활용

In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from scipy.fft import fft, ifft
import pywt
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

### Data Preprocessing

- 5000hz, 3000RPM
- 변환 후 5000hz_cwt_data/3000rpm

In [2]:
RPM = 3000
base_path = "5000hz_raw_data/" + str(RPM) + "rpm/"
folders = [
    str(RPM) + "rpm " + "normal data",
    str(RPM) + "rpm " + "carriage damage",
    str(RPM) + "rpm " + "high-speed damage",
    str(RPM) + "rpm " + "lack of lubrication",
    str(RPM) + "rpm " + "oxidation and corrosion",
]
columns = ["motor1_x", "motor1_y", "motor1_z", "sound","time"]

In [3]:
# 데이터를 읽고 결합하는 함수
def read_and_concatenate(folder):
    all_files = []
    for file_name in os.listdir(folder):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder, file_name)
            df = pd.read_csv(file_path, usecols=columns)
            all_files.append(df)
            break # TEST 용으로 한개만 읽어옴
    combined_df = pd.concat(all_files)
    combined_df.sort_values("time", inplace=True)  # 시간 열 기준 정렬
    return combined_df

### CWT 변환

In [4]:
# CWT를 적용하는 함수
def apply_cwt(data, scales, wavelet_name="morl"):
    coefficients, frequencies = pywt.cwt(data, scales, wavelet_name)
    return coefficients

In [5]:
# CWT 이미지 해석
def plot_cwt(cwt_matrix, time_segment, axis_label, output_dir = "cwt_image"):
    plt.figure(figsize=(10, 4))
    plt.imshow(
        np.abs(cwt_matrix),
        extent=[0, 1, 1, len(cwt_matrix)],
        cmap="viridis",
        aspect="auto",
        interpolation="bilinear",
    )
    plt.colorbar(label="Magnitude")
    plt.title(f"CWT Magnitude for {axis_label} Axis ({time_segment}s Segment)")
    plt.xlabel("Time")
    plt.ylabel("Frequency (Scale)")
    plt.grid(False)
    plt.savefig(f"{output_dir}/{axis_label}_{time_segment}.png")
    plt.close()

In [6]:
# CWT 이미지로 변환
OUTPUT_DIR = "5000hz_cwt_data/" + str(RPM) + "rpm/"
def process_and_save_cwt_images(
    df, folder_name, sampling_rate=5000, segment_length=10, scales=np.arange(1, 129)
):
    # Calculate the number of rows for each segment
    num_rows = sampling_rate * segment_length  # 10 seconds of data

    # Calculate the total number of segments
    time_segments = len(df) // num_rows

    # Loop over each segment
    for index in range(time_segments):
        # Calculate the start and end index for each segment
        start_idx = index * num_rows
        end_idx = start_idx + num_rows

        # Calculate the time segment in seconds
        time_segment = (start_idx / sampling_rate, end_idx / sampling_rate)

        # Extract the x and y data for the current segment
        segment_x = df.iloc[start_idx:end_idx]["motor1_x"]
        segment_y = df.iloc[start_idx:end_idx]["motor1_y"]
        segment_z = df.iloc[start_idx:end_idx]["motor1_z"]
        segment_sound = df.iloc[start_idx:end_idx]["sound"]

        # Apply the Continuous Wavelet Transform (CWT) to the x and y data
        cwtmatr_x = apply_cwt(segment_x, scales)
        cwtmatr_y = apply_cwt(segment_y, scales)
        cwtmatr_z = apply_cwt(segment_z, scales)
        cwtmatr_sound = apply_cwt(segment_sound, scales)

        # Plot and save the CWT results
        plot_cwt(cwtmatr_x, time_segment, "X", OUTPUT_DIR + folder_name)
        plot_cwt(cwtmatr_y, time_segment, "Y", OUTPUT_DIR + folder_name)
        plot_cwt(cwtmatr_z, time_segment, "Z", OUTPUT_DIR + folder_name)
        plot_cwt(cwtmatr_sound, time_segment, "S", OUTPUT_DIR + folder_name)

In [7]:
# 시각화 함수 수정
def visualize_cwt(coefficients, sensor_index, scales):
    plt.figure(figsize=(10, 4))
    plt.imshow(
        np.abs(coefficients[sensor_index]),
        extent=[0, len(coefficients[sensor_index][0]), scales[-1], scales[0]],
        aspect="auto",
        cmap="viridis",
    )
    plt.yscale("log")
    plt.colorbar(label="Magnitude")
    plt.title(f"Continuous Wavelet Transform (CWT) - Sensor {sensor_index + 1}")
    plt.ylabel("Scale")
    plt.xlabel("Time")
    plt.show()

In [8]:
concatenated_df = dict()
folder_index = [
    "normal data",
    "carriage damage",
   "high-speed damage",
    "lack of lubrication",
    "oxidation and corrosion",
]
# 각 폴더에서 데이터를 처리
for index,folder_name in enumerate(folders):
    folder_path = os.path.join(base_path, folder_name)
    concatenated_df[folder_index[index]] = read_and_concatenate(folder_path)

    # time 열 제거
    concatenated_df[folder_index[index]].drop(columns="time", inplace=True)
    # Label 열 추가
    concatenated_df[folder_index[index]]["label"] = index


# # 데이터 결합
combined_data = pd.concat(
    [
        concatenated_df[folder_index[0]],
        concatenated_df[folder_index[1]],
        concatenated_df[folder_index[2]],
        concatenated_df[folder_index[3]],
        concatenated_df[folder_index[4]],
    ],
    ignore_index=True,
)
features = combined_data[["motor1_x", "motor1_y", "motor1_z", "sound"]]
labels = combined_data["label"]

# 데이터 정규화
scalser = StandardScaler()
X_scaled = scalser.fit_transform(features)

In [9]:
BATCH_SIZE = 64
# 데이터를 훈련 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(
    features.values, labels.values, test_size=0.2, random_state=42
)

# 훈련 데이터를 훈련 및 검증 세트로 분할
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

In [10]:
# CWT 변환
def apply_cwt_to_dataset(data, scales, wavelet_name="morl"):
    cwt_features = []
    for feature in data.T:  # Apply CWT on each feature (column) of the dataset
        cwt_matrix, _ = pywt.cwt(feature, scales, wavelet_name)
        # Normalize the CWT matrix
        cwt_matrix = (cwt_matrix - np.mean(cwt_matrix)) / np.std(cwt_matrix)
        cwt_features.append(cwt_matrix)
    # Stack to form [samples, features, time, CWT_coefficients]
    return np.stack(cwt_features, axis=1)

In [12]:
# Convert CWT
scales = np.arange(1, 128)
X_train_cwt = apply_cwt_to_dataset(X_train, scales)
X_val_cwt = apply_cwt_to_dataset(X_val, scales)
X_test_cwt = apply_cwt_to_dataset(X_test, scales)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_cwt, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_cwt, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_cwt, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

print(
    X_train_tensor.shape
)  # Expected to be (number of samples, features, CWT coefficients)
print(y_train_tensor.shape)  # Expected to be (number of samples,)

torch.Size([127, 4, 960000])
torch.Size([960000])


In [ ]:
# PyTorch의 Dataset 및 DataLoader 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### LSTM 모델 구현

In [ ]:
# LSTM 오토인코더 모델 정의
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMEncoder, self).__init__()
        self.encoder_lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True
        )
        self.decoder_lstm = nn.LSTM(
            hidden_size, input_size, num_layers, batch_first=True
        )

    def forward(self, x):
        encoded_output, _ = self.encoder_lstm(x)
        decoded_output, _ = self.decoder_lstm(encoded_output)
        return decoded_output

In [ ]:
# Hyperparameters
input_size = X_train.shape[2] # ["motor1_x", "motor1_y", "motor1_z", "sound"]
hidden_size = 128
num_layers = 2
learning_rate = 0.001
num_epochs = 100

In [ ]:
# 모델 초기화
model = LSTMEncoder(input_size, hidden_size, num_layers)

# Loss function 및 optimizer 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Training

In [ ]:
# 체크포인트 파일 경로 설정
checkpoint_path = "./checkpoint/cwt_lstm_autoencoder/model_checkpoint"

# 모델 학습
best_val_loss = float("inf")
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for batch_x, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_x)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_x.size(0)

        # 정확도 계산
        _, predicted = torch.max(output, 1)
        total_train += batch_y.size(0)
        correct_train += (predicted == batch_y).sum().item()

    # Train Loss & Accuracy 계산
    train_loss /= len(train_loader.dataset)
    train_accuracy = correct_train / total_train

    # 검증 세트로 모델 평가
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            output = model(batch_x)
            loss = criterion(output, batch_x)
            val_loss += loss.item() * batch_x.size(0)

            # 정확도 계산
            _, predicted = torch.max(output, 1)
            total_val += batch_y.size(0)
            correct_val += (predicted == batch_y).sum().item()

    # Validation Loss & Accuracy 계산
    val_loss /= len(val_loader.dataset)
    val_accuracy = correct_val / total_val

    # 모델의 체크포인트 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        # 체크포인트 저장
        torch.save(model.state_dict(), checkpoint_path+str(epoch)+".pth")
        print("Checkpoint saved.")

    if (epoch + 1) % 1 == 0:
        print(
            f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}"
        )

### Testing

In [ ]:
# 체크포인트 불러오기
model.load_state_dict(torch.load(checkpoint_path))

model.eval()
test_loss = 0.0
correct_test = 0
total_test = 0
with torch.no_grad():
    for batch_x, batch_y in tqdm(test_loader, desc="Testing"):
        output = model(batch_x)
        loss = criterion(output, batch_x)
        test_loss += loss.item() * batch_x.size(0)

        # 정확도 계산
        _, predicted = torch.max(output, 1)
        total_test += batch_y.size(0)
        correct_test += (predicted == batch_y).sum().item()

# Test Loss & Accuracy 계산
test_loss /= len(test_loader.dataset)
test_accuracy = correct_test / total_test

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")